In [4]:
from src.gen import Gen
from src.puzzle import Nonogram
from src.state import State
from src.search import DFS, BeFS
from src.utils import heuristic_level


grid = [
    [1, 1, 1, 0, 1, 0, 1],
    [1, 1, 1, 0, 1, 1, 0],
    [0, 0, 1, 1, 1, 1, 1],
    [1, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 1],
    [0, 1, 0, 1, 1, 0, 1],
    [1, 0, 1, 1, 0, 0, 1],
]
# grid = [
#     [1, 1, 0, 1],
#     [0, 1, 0, 0],
#     [0, 1, 1, 0],
#     [1, 0, 0, 1]
# ]
# grid = [
#     [1, 0, 1],
#     [0, 0, 1],
#     [0, 1, 1]
# ]
# grid = [
#     [1, 0],
#     [0, 1]
# ]
init = State(len(grid), num=Gen.gen_grid_num(grid))
# init = State(size=7)
puzzle = Nonogram(init)

In [5]:
print(DFS(puzzle).path())

[<Node 
<State level=None remain={0, 1, 2, 3, 4, 5, 6}
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 

0  0  0  0  0  0  0  

>
>, <Node 
<State level=6 remain={0, 1, 2, 3, 4, 5}
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 

0  0  0  0  0  0  0  

>
>, <Node 
<State level=6 remain={0, 1, 2, 3, 4, 5}
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  #  .  .  .  .  .    1 

0  1  0  0  0  0  0  

>
>, <Node 
<State level=6 remain={0, 1, 2, 3, 4, 5}
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  #  .  #  

In [6]:
print(BeFS(puzzle,heuristic_level).path())

[<Node 
<State level=None remain={0, 1, 2, 3, 4, 5, 6}
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 

0  0  0  0  0  0  0  

>
>, <Node 
<State level=2 remain={0, 1, 3, 4, 5, 6}
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 

0  0  0  0  0  0  0  

>
>, <Node 
<State level=2 remain={0, 1, 3, 4, 5, 6}
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  #  #  #  #  #    5 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 

0  0  1  1  1  1  1  

>
>, <Node 
<State level=0 remain={1, 3, 4, 5, 6}
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  #  #  #  #  #    5 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  